In [1]:
import json
import pandas as pd
import numpy as np
import os
import time
import requests
import logging

logic: 
1. at the last block of current epoch, get the address of all the eligible validators
2. at the first block of next epoch, group all the eligible validators into elected and non-elected
3. sort elected and non-elected groups based on their stakes
4. compare the lowest stake in elected group and highest stake in non-elected group. 
expectation: lowest stake in elected group > highest stake in non-elected group

Question: if two validators have the same stake and both active, their stake rank is No.200. Who will be elected? based on the order who joined the network first?

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    return block, last_block

In [7]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result']

In [8]:
def getEligibleValidator():
    eligible = []
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['epos-status'] == 'currently elected and signing enough blocks to be eligible for election next epoch' or\
        i['epos-status'] == 'eligible to be elected next epoch':
            address = i['validator']['address']
            eligible.append(address)
    return eligible

In [9]:
def getValidatorInfo(validator):
    method = "hmy_getValidatorInformation"
    params = [validator]
    return get_information(method, params)['result']

In [10]:
def getStake(eligible):
    elected = dict()
    non_elected = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        address = i['validator']['address']
        if address in eligible:
            stake = i['total-delegation']
            if i['currently-in-committee'] == True:
                elected[address] = stake
            else:
                non_elected[address] = stake
    return elected, non_elected

In [11]:
def E4_test(single):
    logger.info(f"Test-E4: Low staker will never get elected over high staker")
    # the number of epoches you want to test
    num = 1
    iterations = 0
    flag = True
    try:
        while iterations < num:
            block, last_block = getCurrentAndLastBlock()
            logger.info(f"current and last block numbers: {block}, {last_block}")
            if block == last_block:
                logger.info(f"current block is the last block in epoch, waiting for the new epoch...")
                new_block = block+1
                while block < new_block:
                    block = getBlockNumber()
                block, last_block = getCurrentAndLastBlock()
                logger.info(f"current and last block numbers: {block}, {last_block}")
            second_last_block = last_block - 1
            while block < second_last_block:
                block = getBlockNumber()
            logger.info(f"second last block in current epoch reached, {block}, wait for 6 seconds to reach the end of the block")
            time.sleep(6)
            logger.info(f"begin collecting eligible validators...")
            validator_infos = getAllValidatorInformation()
            eligible_stake = dict()
            for i in validator_infos:
                if i['epos-status'] == 'currently elected' or i['epos-status'] == 'eligible to be elected next epoch':
                    address = i['validator']['address']
                    eligible_stake[address] = i['total-delegation']
            # reach the first block in next epoch and check the status of all eligible validators
            new_epoch_block = block + 1
            while block < new_epoch_block:
                block = getBlockNumber()
            logger.info(f"first block of new epoch reached, {new_epoch_block}, will begin checking all the elgible validators' election result...")
            elected = dict()
            non_elected = dict()
            validator_infos = getAllValidatorInformation()
            for i in validator_infos:
                if i['metrics']:
                    address = i['validator']['address']
                    by_key_metrics = i['metrics']['by-bls-key']
                    slots = len(by_key_metrics)
                    if address in eligible_stake:
                        if i['currently-in-committee']:
                            elected[address] = float(eligible_stake[address] / slots)
                        else:
                            non_elected[address] = float(eligible_stake[address] / slots)
            sorted_elected = sorted(elected.items(), key = lambda kv: kv[1])
            sorted_non_elected = sorted(non_elected.items(), key = lambda kv: kv[1], reverse = True)

            # get the lowest elected validator and highest non-elected validator
            if not sorted_elected:
                lowest_elected = 0
            else:
                lowest_elected = sorted_elected[0][1]
            if not sorted_non_elected:
                highest_unelected = 0
            else:
                highest_unelected = sorted_non_elected[0][1]
            if lowest_elected < highest_unelected:
                logger.warning(f"Test-E4: Fail")
                logger.warning(f"lowest stake in elected eligible validators: {sorted_elected[0]}" )
                logger.warning(f"highest stake in unelected eligible validators: {sorted_non_elected[0]}\n")
                flag = False
            iterations += 1
    except TypeError as e:
        logger.error(f"error: {e}")
    curr_test = None
    if flag:
        logger.info(f"Test-E4: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [12]:
E4_test(True)

INFO:economic-test:Test-E4: Low staker will never get elected over high staker
INFO:economic-test:current and last block numbers: 3470277, 3473407
INFO:economic-test:second last block in current epoch reached, 3473406, wait for 6 seconds to reach the end of the block
INFO:economic-test:begin collecting eligible validators...
INFO:economic-test:first block of new epoch reached, 3473407, will begin checking all the elgible validators' election result...
INFO:economic-test:Test-E4: Succeed



(True, None)